In [12]:
import cv2
import os
import glob
import pandas as pd
import math

import numpy as np
# from scipy.misc import imresize
from imageio import imread, imsave
from natsort import natsorted

# from labelConv import label2int, int2label

In [13]:
def get_contour_precedence(contour, cols):
    tolerance_factor = 10
    origin = cv2.boundingRect(contour)
    return ((origin[1] // tolerance_factor) * tolerance_factor) * cols + origin[0]

In [14]:
def get_contour_precedence_two(contour, cols):
    tolerance_factor = 10
    origin = cv2.boundingRect(contour)
    return ((origin[0] // tolerance_factor) * tolerance_factor) * cols + origin[1]

In [15]:
def captch_ex(file_name):
    img = cv2.imread(file_name)
    img = cv2.resize(img,None,fx=0.3, fy=0.3, interpolation = cv2.INTER_CUBIC)
        
    img_final = cv2.imread(file_name)
    img2gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, mask = cv2.threshold(img2gray, 180, 255, cv2.THRESH_BINARY)
    image_final = cv2.bitwise_and(img2gray, img2gray, mask=mask)
    ret, new_img = cv2.threshold(image_final, 180, 255, cv2.THRESH_BINARY)  # for black text , cv.THRESH_BINARY_INV
    '''
            line  8 to 12  : Remove noisy portion 
    '''
    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (3,
                                                         3))  # to manipulate the orientation of dilution , large x means horizonatally dilating  more, large y means vertically dilating more
    dilated = cv2.dilate(new_img, kernel, iterations=5)  # dilate , more the iteration more the dilation

    # for cv2.x.x

    _, contours,hierarchy = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)  # get contours
    
    #print(contours[0].shape)
    contours.sort(key=lambda x:get_contour_precedence(x, img.shape[1]))
    
    # for cv3.x.x comment above line and uncomment line below

    #image, contours, hierarchy = cv2.findContours(dilated,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)

    index = 1
    for contour in contours:
        # get rectangle bounding contour
        [x, y, w, h] = cv2.boundingRect(contour)

        if(w*h < 800):
            continue
        # draw rectangle around contour on original image
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 255), 1)

        
        #you can crop image and send to OCR  , false detected will return no text :)
        cropped = img[y :y +  h , x : x + w]
        #print(file_name.split('.')[0])
        s = 'data/crop_' + str(index) + '.jpg' 
        #print(s)
        cv2.imwrite(s , cropped)
        #cv2.imshow('captcha_result', cropped)
        #cv2.waitKey()
        index = index + 1

        
    #print(index-1)
    # write original image with added contours to disk
    #cv2.imshow('captcha_result', img)
    #cv2.waitKey()
    return index-1

In [16]:
def captch_ex2(file_name, index):
    img = cv2.imread(file_name)
    img = cv2.resize(img,None,fx=1, fy=1, interpolation = cv2.INTER_CUBIC)
        
    img_final = cv2.imread(file_name)
    img2gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, mask = cv2.threshold(img2gray, 180, 255, cv2.THRESH_BINARY)
    image_final = cv2.bitwise_and(img2gray, img2gray, mask=mask)
    ret, new_img = cv2.threshold(image_final, 180, 255, cv2.THRESH_BINARY)  # for black text , cv.THRESH_BINARY_INV
    '''
            line  8 to 12  : Remove noisy portion 
    '''
    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (1,
                                                         1))  # to manipulate the orientation of dilution , large x means horizonatally dilating  more, large y means vertically dilating more
    dilated = cv2.dilate(new_img, kernel, iterations=0)  # dilate , more the iteration more the dilation

    # for cv2.x.x

    _, contours,hierarchy = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)  # get contours

    # for cv3.x.x comment above line and uncomment line below

    #image, contours, hierarchy = cv2.findContours(dilated,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    contours.sort(key=lambda x:get_contour_precedence_two(x, img.shape[1]))
    index2 = 1
    for contour in contours:
        # get rectangle bounding contour
        [x, y, w, h] = cv2.boundingRect(contour)

        # draw rectangle around contour on original image
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 255), 1)

        
        #you can crop image and send to OCR  , false detected will return no text :)
        cropped = img[y :y +  h , x : x + w]
        #print(file_name.split('.')[0])
        s = 'data/test/crop_' + str(index) + '_' + str(index2) + '.jpg' 
        #print(s)
        cv2.imwrite(s , cropped)
        #cv2.imshow('captcha_result', cropped)
        #cv2.waitKey()
        index2 = index2 + 1

        
    #print(index2-1)
    # write original image with added contours to disk
    #cv2.imshow('captcha_result', img)
    #cv2.waitKey()
    return index2-1

In [17]:
def clearFolder(dirPath):
    fileList = os.listdir(dirPath)
    for fileName in fileList:
        os.remove(dirPath+"/"+fileName)
    return

In [20]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, Conv2D

Using TensorFlow backend.
/home/shyam/MyCode/text_recognition/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/shyam/MyCode/text_recognition/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/shyam/MyCode/text_recognition/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/h

In [21]:
# Input image dimensions
img_rows, img_cols = 20, 20
batch_size = 128
nb_classes = 62  # A-Z, a-z and 0-9
nb_epoch = 500
model = Sequential()

model.add(Conv2D(128, (3, 3), padding='same', kernel_initializer='he_normal', activation='relu', input_shape=(img_rows, img_cols, 1)))
model.add(Conv2D(128, (3, 3), padding='same', kernel_initializer='he_normal', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), padding='same', kernel_initializer='he_normal', activation='relu'))
model.add(Conv2D(256, (3, 3), padding='same', kernel_initializer='he_normal', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(512, (3, 3), padding='same', kernel_initializer='he_normal', activation='relu'))
model.add(Conv2D(512, (3, 3), padding='same', kernel_initializer='he_normal', activation='relu'))
model.add(Conv2D(512, (3, 3), padding='same', kernel_initializer='he_normal', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(4096, kernel_initializer='he_normal', activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, kernel_initializer='he_normal', activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, kernel_initializer='he_normal', activation='softmax'))

#model.summary()






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [22]:
model.load_weights("best.kerasModelWeights")

OSError: Unable to open file (unable to open file: name = 'best.kerasModelWeights', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
file_name = 'sample.jpg'
num_images = captch_ex(file_name)
#num = captch_ex2('data/crop_8.jpg', 8)
clearFolder('data/test')
records = []

for i in range(num_images):
    path = 'data/crop_'
    new_filename = path + str(i+1) + '.jpg'
    num_char = captch_ex2(new_filename, index=i+1)
    records.append((i+1, num_char))
    
records

In [ ]:
def preprocessing(FILEPATH):

    
    #path = "data"
    path = FILEPATH + '/test'
    # Input image dimensions
    img_rows, img_cols = 20, 20

    # Keep or not the initial image aspect ratio
    keepRatio = False

    # Suffix of the created directories and files
    suffix = "Preproc_" + str(img_rows) + "_" + str(img_cols) + ("_kr" if keepRatio else "")


    # We have to make sure files are sorted according to labels, even if they don't have trailing zeros
    files = natsorted(glob.glob(path + "/*"))

    data = np.zeros((len(files), img_rows, img_cols))  # will add the channel dimension later

    for i, filepath in enumerate(files):
        #print(filepath)
        ext = filepath.split('.')[1]
        if(ext == "db"):
            i-=1
            break

        image = imread(filepath, True)  # True: flatten to grayscale
        if keepRatio:
            # Find the largest dimension (height or width)
            maxSize = max(image.shape[0], image.shape[1])

            # Size of the resized image, keeping aspect ratio
            imageWidth = math.floor(img_rows * image.shape[0] / maxSize)
            imageHeigh = math.floor(img_cols * image.shape[1] / maxSize)

            # Compute deltas to center image (should be 0 for the largest dimension)
            dRows = (img_rows - imageWidth) // 2
            dCols = (img_cols - imageHeigh) // 2

            imageResized = np.zeros((img_rows, img_cols))
            imageResized[dRows:dRows + imageWidth, dCols:dCols + imageHeigh] = imresize(image, (imageWidth, imageHeigh))

            # Fill the empty image with the median value of the border pixels
            # This value should be close to the background color
            val = np.median(np.append(imageResized[dRows, :],
                                      (imageResized[dRows + imageWidth - 1, :],
                                       imageResized[:, dCols],
                                       imageResized[:, dCols + imageHeigh - 1])))

            # If rows were left blank
            if (dRows > 0):
                imageResized[0:dRows, :].fill(val)
                imageResized[dRows + imageWidth:, :].fill(val)

            # If columns were left blank
            if (dCols > 0):
                imageResized[:, 0:dCols].fill(val)
                imageResized[:, dCols + imageHeigh:].fill(val)
        else:
            imageResized = imresize(image, (img_rows, img_cols))

        # Add the resized image to the dataset
        data[i] = imageResized

        # Save image (mostly for visualization)
        filename = filepath.split("\\")[-1]
        filenameDotSplit = filename.split(".")
        newFilename = filenameDotSplit[0].zfill(5) + "." + filenameDotSplit[-1].lower()  # Add trailing zeros
        newName = "/".join(filepath.split("\\")[:-1]) + "/" + newFilename
        imsave(newName, imageResized)

    # Add channel/filter dimension
    data = data[:, :, :, np.newaxis]

    # Makes values floats between 0 and 1 (gives better results for neural nets)
    data = data.astype('float32')
    data /= 255
    print(data.shape)
    # Save the data as numpy file for faster loading
    np.save(FILEPATH + "/" + suffix + ".npy", data)
    
    return

In [ ]:
preprocessing('data')

In [ ]:
path_to_npy_file = 'data/Preproc_20_20.npy'

In [ ]:
def CNN(model, path_to_npy_file, records):
    X = np.load(path_to_npy_file)
    Y = model.predict_classes(X)
    
    # Translate integers to character labels
    vInt2label = np.vectorize(int2label)
    Y = vInt2label(Y)
    
    list_of_words = []
    k = 0
    for i in range(len(records)):
        s = ''
        for j in range(records[i][1]):
            c = Y[k].lower()
            if(c == 'j'):
                s += ''
            else:
                s += c
            k += 1
        if(i in [5,6,7,9,10,11,16,17,18,19,20,22,23]):
            list_of_words.append(s.lower())
    
    return list_of_words

In [ ]:
list_of_words = CNN(model, path_to_npy_file, records)
len(list_of_words)

In [ ]:
list_of_words